In [1]:
import random
import numpy as np
from IPython.display import display, HTML

In [2]:
def is_power_of_two(n):
    return (n != 0) and (n & (n-1) == 0)
    
def display_sbs(*args):
    'display side by side'
    ''
    html_str = '''\
<div style="display: flex; gap: 2em; align-items: flex-start">
{}
</div>'''.format('\n'.join(arg._repr_html_() for arg in args))

    display(HTML(html_str))

    

class DNF:
    def __init__(self, masks):
        '''
        masks: ([0, 0, 0, 0], [0, 1, 1, 0], ...)
        '''
        self.n = len(masks[0])
        self.masks = masks
        
    def __call__(self, arg):
        '''
        Dычисляет значение DNF на векторе x
        
        arg: int
        '''
        n = self.n
        x = [int(i) for i in f'{arg:0{n}b}'] # бинарное представление числа arg ширины n
        return any(
            all(arg if bit else not arg for arg, bit  in zip(x, mask))
            for mask in self.masks
        )
    
    def to_function(self):
        '''
        Преобразует DNF в BFunction
        '''
        return BFuncion(
            [self(i) for i in range(2**self.n)]
        )
        
    @classmethod
    def from_string(cls, string):
        return cls([
            [
                0 if var.strip()[0] == '~' else 1
                for var in s.split('&')
            ]
            for s in string.split('|')
        ])
        
    def _repr_latex_(self):
        # 0110 -> \overline{x}_0 x_1 x_2 \overline{x}_3
        n = self.n
        return '$' + ' \lor '.join(
            ' '.join(
                f'x_{k}' if x else f'\\overline{{x}}_{k}'
                for k, x in enumerate(mask)
            )
            for mask in self.masks
        ) + '$'

class CNF:
    def __init__(self, masks):
        '''
        masks: ([0, 0, 0, 0], [0, 1, 1, 0], ...)
        '''
        self.n = len(masks[0])
        self.masks = masks 

    def __call__(self, arg):
        '''
        Dычисляет значение CNF на векторе x
        
        arg: int
        '''
        n = self.n
        x = [int(i) for i in f'{arg:0{n}b}'] # бинарное представление числа arg ширины n
        return all(
            any(arg if bit else not arg for arg, bit  in zip(x, mask))
            for mask in self.masks
        )

    def to_function(self):
        '''
        Преобразует DNF в BFunction
        '''
        return BFuncion(
            [self(i) for i in range(2**self.n)]
        )

    @classmethod
    def from_string(cls, string):
        return cls([
            [
                0 if var.strip()[0] == '~' else 1
                for var in s.strip(' )(').split('|')
            ]
            for s in string.split('&')
        ])

    def _repr_latex_(self):
        # 0110 -> \overline{x}_0 x_1 x_2 \overline{x}_3
        n = self.n
        return '$' + ' \land '.join(
            '({})'.format(' \lor '.join(
                f'x_{k}' if x else f'\\overline{{x}}_{k}'
                for k, x in enumerate(mask)
            ))
            for mask in self.masks
        ) + '$'

   
    
class BFuncion:
    def __init__(self, values):
        '''
        values: [0, 1, ..] или numpy массив
            вектор значений функции
        n: int
            арность функции
        nf: int
            колличество записей в таблице истинности
        '''
        len_values = len(values)
        assert is_power_of_two(len_values), 'Length not power of 2' # проверка на степень двойки
        self.n  = len_values.bit_length() - 1
        self.nf = len_values
        self.values = np.array(values, dtype=np.bool_)

    def __call__(self, arg):
        '''
        arg: int или массив numpy длины n из bool
            аргумент функции
        '''
        n = self.n
        if isinstance(arg, int):
            idx = arg
        else:
            idx = np.inner(arg, np.logspace(n-1, 0, num=n, base=2, dtype=int))

        return self.values[idx]
    
    def __eq__(self, other):
        'Проверка функций на равенство f1 == f2'
        assert isinstance(other, type(self))
        return np.array_equal(self.values, other.values)

    @classmethod
    def random(cls, n):
        '''
        Генерирует случайную функцию от n переменных

        n : int
            колличество аргументов функции
        '''
        rng = np.random.default_rng()
        values = rng.integers(0, 2, 2**n, dtype=np.bool_)
        return cls(values)

    def partial(self, sigma, index):
        '''
        Получить остаточную функцию

        sigma: bool, 0 or 1
            значение переменной для остаточной функции

        index: int, [0; n)
            индекс переменной в промежутке [0, n)
        '''
        rindex = self.n - 1 - index
        mask = 1 << rindex
        return type(self)([
            v
            for i, v in enumerate(self.values)
            if (i & mask) >> rindex == sigma
        ])

    def dnf(self):
        '''
        Построение СДНФ
        '''
        n = self.n
        return DNF([
            [int(b) for b in f'{i:0{n}b}'] # список из битов i
            for i in range(self.nf)
            if f(i) == 1
        ])

    def cnf(self):
        '''
        Построение СКНФ
        '''
        n = self.n
        return CNF([
            [int(b) for b in f'{i ^ ((1 << n) - 1):0{n}b}'] # список из инвертируемых битов i
            for i in range(self.nf)
            if f(i) == 0
        ])
        
    # def __str__(self):
    #     return '\n'.join([
    #         '|'.join([f'x{i}' for i in range(1, n+1)] + ['-> f(x)']),
    #         '|'.join(':---:' for _ in range(n + 1)),
    #         *[
    #             '|'.join(list(f'{i:0{n}b}') + [f'-> {self(i):b}'])
    #             for i in range(nf)
    #         ],
    #     ])
    
    def _repr_html_(self):
        '''
        представление функции в виде таблички
        '''
        n = self.n
        nf = self.nf
        tab = ' ' * 2
        nl = '\n'
        return '\n'.join([
'''\
<style>
  td {
      text-align: center !important;
  }
    
  td:last-child {
    border-left: solid 1px var(--jp-ui-font-color3) !important;
  }
</style>
''',
            '<table>',
            f'{tab}<thead>',
            f'''{2*tab}<tr>\n{
                nl.join(
                    [f'{3*tab}<th>x<sub>{i}</sub></th>' for i in range(n)] +
                    [f'{3*tab}<th>f(x)</th>']
                )
            }\n{2*tab}</tr>''',
            f'{tab}</thead>',
            f'{tab}<tbody>',
            *[
                f'''{2*tab}<tr>\n{
                    nl.join(
                        [f'{3*tab}<td>{x}</td>' for x in f'{i:0{n}b}'] +
                        [f'{3*tab}<td>{self(i):b}</td>']
                    )
                }\n{2*tab}</tr>'''
                for i in range(nf)
            ],
            f'{tab}</tbody>',
            '</table>',
        ])

def bfunction_from_partials(values0, values1, index):
    '''
    Создание булевой функции из нулевой и единичной остаточной

    values0: [0, 1, ..] или numpy массив 
        нулевая остаточная

    values1: [0, 1, ..] или numpy массив 
        единичная остаточная

    index: int
        индекс переменной в промежутке [0, n)
    '''
    nf = 2 * len(values0)
    n = nf.bit_length() - 1
    assert index < n
    rindex = n - 1 - index
    mask = 1 << rindex
    values = [values0, values1]
    return BFuncion([
        values[(i & mask) >> rindex][(i >> (rindex + 1) << rindex) + i % mask]
        for i in range(nf)
    ])

## 1. На вход число n, на выход функция от n аргументов.

In [3]:
n = 4
f = BFuncion.random(4)
f

x0,x1,x2,x3,f(x)
0,0,0,0,1
0,0,0,1,1
0,0,1,0,0
0,0,1,1,1
0,1,0,0,0
0,1,0,1,0
0,1,1,0,1
0,1,1,1,1
1,0,0,0,1
1,0,0,1,0


## 2. На вход вектор функции, 0 или 1, номер аргумента, на выход соответствующая остаточная.

In [4]:
display_sbs(
    f,
    f.partial(0, index=1), # 0 остаточная по индексу 1 
    f.partial(1, index=1), # 1 остаточная по индексу 1 
)

x0,x1,x2,x3,f(x)
0,0,0,0,1
0,0,0,1,1
0,0,1,0,0
0,0,1,1,1
0,1,0,0,0
0,1,0,1,0
0,1,1,0,1
0,1,1,1,1
1,0,0,0,1
1,0,0,1,0


# 3. На вход два вектора функции (это нулевая и единичная остаточные), <br>номер аргумента, на выход соответствующая функция.

In [5]:
f0 = f.partial(0, index=1)
f1 = f.partial(1, index=1)

display_sbs(
    f,
    f0,
    f1,
    bfunction_from_partials(f0.values, f1.values, 1)
)

x0,x1,x2,x3,f(x)
0,0,0,0,1
0,0,0,1,1
0,0,1,0,0
0,0,1,1,1
0,1,0,0,0
0,1,0,1,0
0,1,1,0,1
0,1,1,1,1
1,0,0,0,1
1,0,0,1,0


## 4. Игра. Узнать имя функции от 2-х аргументов. <br>Система предлагает вектор функции, пользователь выбирает «имя» (одно из 16).

In [6]:
bf2_names = {
    'тождественный ноль'        : (0, 0, 0, 0),
    'конъюнкция'                : (0, 0, 0, 1),
    'коимпликация'              : (0, 0, 1, 0),
    'первый операнд'            : (0, 0, 1, 1),
    'меньше'                    : (0, 1, 0, 0),
    'второй операнд'            : (0, 1, 0, 1),
    'сложение'                  : (0, 1, 1, 0),
    'дизъюнкция'                : (0, 1, 1, 1),
    'стрелка Пирса'             : (1, 0, 0, 0),
    'эквивалентность'           : (1, 0, 0, 1),
    'отрицание второго операнда': (1, 0, 1, 0),
    'обратная импликация'       : (1, 0, 1, 1),
    'отрицание первого операнда': (1, 1, 0, 0),
    'импликация'                : (1, 1, 0, 1),
    'штрих Шеффера'             : (1, 1, 1, 0),
    'тождественная единица'     : (1, 1, 1, 1),
}

name, values = random.choice(list(bf2_names.items()))
print('Угадай имя функции')
display(BFuncion(values))

answer = input()
answer = ' '.join(answer.strip().lower().split())
print('Верно' if answer == name else 'Не верно')

Угадай имя функции


x0,x1,f(x)
0,0,0
0,1,0
1,0,0
1,1,1


 конъюнкция


Верно


# 5. Игра. Существенные и фиктивные переменные. Система предлагает вектор функции. <br>Пользователь выбирает существенные и фиктивные переменные.

In [7]:
f = BFuncion.random(4)
display(f)
i = int(input('Введите индекс существенной переменной\n'))

assert i < f.n, f'index more then {f.n - 1}'
answer = 'Не верно. Это несущественная переменная' if f.partial(0, i) == f.partial(1, i) else 'Верно'
display_sbs(f.partial(0, i), f.partial(1, i))
print(answer)

x0,x1,x2,x3,f(x)
0,0,0,0,0
0,0,0,1,0
0,0,1,0,0
0,0,1,1,1
0,1,0,0,1
0,1,0,1,1
0,1,1,0,0
0,1,1,1,0
1,0,0,0,1
1,0,0,1,0


Введите индекс существенной переменной
 0


x0,x1,x2,f(x)
0,0,0,0
0,0,1,0
0,1,0,0
0,1,1,1
1,0,0,1
1,0,1,1
1,1,0,0
1,1,1,0
x0,x1,x2,f(x)
0,0,0,1


Верно


## 6. Игра. ДНФ. Система предлагает вектор функции. <br>Пользователь вводит ДНФ. Система определяет правильно или нет.

In [8]:
# ~x0 & ~x1 & ~x2 | x0 & x1 & x2
f = BFuncion([1, 0, 0, 0, 0, 0, 0, 1])
display(f)
dnf_string = input('Введите ДНФ в виде x0 & ~x1 & x2 | ~x0 & ~x1 & x2 | ..\n')

dnf = DNF.from_string(dnf_string)
display(dnf)
answer = 'Верно' if dnf.to_function() == f else 'Не верно'
print(answer)

x0,x1,x2,f(x)
0,0,0,1
0,0,1,0
0,1,0,0
0,1,1,0
1,0,0,0
1,0,1,0
1,1,0,0
1,1,1,1


Введите ДНФ в виде x0 & ~x1 & x2 | ~x0 & ~x1 & x2 | ..
 ~x0 & ~x1 & ~x2 | x0 & x1 & x2


Верно


## 7. Игра. КНФ. Система предлагает вектор функции. <br>Пользователь вводит КНФ. Система определяет правильно или нет.

In [10]:
# (x0 | x1 | x2) & (x0 | x1 | ~x2) & (x0 | ~x1 | x2) & (~x0 | x1 | ~x2)
f = BFuncion([0, 0, 0, 1, 1, 0, 1, 1])
display(f)
сnf_string = input('Введите КНФ в виде (x0 | ~x1 | x2) & (~x0 | ~x1 | x2) & ..\n')

сnf = CNF.from_string(сnf_string)
display(сnf)
answer = 'Верно' if сnf.to_function() == f else 'Не верно'
print(answer)

x0,x1,x2,f(x)
0,0,0,0
0,0,1,0
0,1,0,0
0,1,1,1
1,0,0,1
1,0,1,0
1,1,0,1
1,1,1,1


Введите КНФ в виде (x0 | ~x1 | x2) & (~x0 | ~x1 | x2) & ..
 (x0 | x1 | x2) & (x0 | x1 | ~x2) & (x0 | ~x1 | x2) & (~x0 | x1 | ~x2)


Верно


## 8. Пользователь вводит вектор функции. Система строит СДНФ.

In [11]:
# 1 0 1 0 1 0 0 1
vstring = input('Введите вектор функции, например 1, 0, 1 ..\n')
f = BFuncion([int(i) for i in vstring.split()])
display(f)
f.dnf()

Введите вектор функции, например 1, 0, 1 ..
 1 0 1 0 1 0 0 1


x0,x1,x2,f(x)
0,0,0,1
0,0,1,0
0,1,0,1
0,1,1,0
1,0,0,1
1,0,1,0
1,1,0,0
1,1,1,1


## 9. Пользователь вводит вектор функции. Система строит СКНФ.

In [12]:
# 1 1 1 0 0 0 1 0 0 0 1 0 1 1 1 0
vstring = input('Введите вектор функции, например 1, 0, 1 ..\n')
f = BFuncion([int(i) for i in vstring.split()])
display(f)
f.cnf()

Введите вектор функции, например 1, 0, 1 ..
 1 1 1 0 0 0 1 0 0 0 1 0 1 1 1 0


x0,x1,x2,x3,f(x)
0,0,0,0,1
0,0,0,1,1
0,0,1,0,1
0,0,1,1,0
0,1,0,0,0
0,1,0,1,0
0,1,1,0,1
0,1,1,1,0
1,0,0,0,0
1,0,0,1,0


## 10. Игра. Замкнутые классы б.ф. Система предлагает вектор функции. <br>Пользователь должен выбрать замкнутые классы, которым эта функция принадлежит. <br>Система определяет правильно выбраны классы или нет.